<a href="https://colab.research.google.com/github/selamawitsh/MachineLearning_lab_assignment_1/blob/main/fraud_training_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib


In [ ]:
df = pd.read_csv("/content/AIML Dataset.csv")
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Drop ID-like columns
df = df.drop(["nameOrig", "nameDest", "isFlaggedFraud"], axis=1, errors='ignore')

# Encode categorical feature 'type'
categorical_cols = ["type"]
numeric_cols = df.drop(["type", "isFraud"], axis=1).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numeric_cols)
    ]
)

# Split data
X = df.drop("isFraud", axis=1)
y = df["isFraud"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

dt_model = DecisionTreeClassifier(
    random_state=42,
    class_weight='balanced',
    max_depth=10,           # allow deeper tree
    min_samples_leaf=5      # prevent pruning rare fraud
)
dt_model.fit(X_train_preprocessed, y_train)

dt_pred = dt_model.predict(X_test_preprocessed)
print("\n==== Decision Tree Report ====")
print(classification_report(y_test, dt_pred))




==== Decision Tree Report ====
              precision    recall  f1-score   support

           0       1.00      0.99      1.00   1270904
           1       0.13      0.99      0.23      1620

    accuracy                           0.99   1272524
   macro avg       0.57      0.99      0.62   1272524
weighted avg       1.00      0.99      0.99   1272524



In [ ]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(max_iter=300, class_weight='balanced')
log_model.fit(X_train_preprocessed, y_train)
log_pred = log_model.predict(X_test_preprocessed)
print("\n==== Logistic Regression Report ====")
print(classification_report(y_test, log_pred))



==== Logistic Regression Report ====
              precision    recall  f1-score   support

           0       1.00      0.95      0.97   1270904
           1       0.02      0.95      0.05      1620

    accuracy                           0.95   1272524
   macro avg       0.51      0.95      0.51   1272524
weighted avg       1.00      0.95      0.97   1272524



In [ ]:
import joblib

joblib.dump(dt_model, "decision_tree_model.pkl")
joblib.dump(log_model, "logistic_regression_model.pkl")
joblib.dump(preprocessor, "preprocessor.pkl")

print("Models exported successfully!")


Models exported successfully!
